### The difference between this notebook and `Pre-processing` is that some information is _missing_ here, so I would advise to go in `Pre-processing` instead. Aside from a few comments the code and other contents is the same

In [5]:
# Setup the package, this is done automatically with the startup scripts
import os
import sys

# Setup some constants
HOME = os.path.expanduser('~')
CHADWICK_PATH = os.path.join(HOME, 'win_img', 'chadwick_insert')
CHADWICK_SAMPLE_PATH = os.path.join(CHADWICK_PATH, 'Data')
CHADWICK_DARK_PATH = os.path.join(CHADWICK_PATH, 'Dark_Tomo')
CHADWICK_FLAT_PATH = os.path.join(CHADWICK_PATH, 'Open_Tomo_BeforeInsert')

MANTIDIMAGING_PATH = os.path.join(HOME, 'mantidimaging')
print("Home Dir ->", HOME, "\nDataset Path ->", CHADWICK_PATH, "\nPackage Path ->", MANTIDIMAGING_PATH)
# Append the ISIS Imaging path so that Python can find the package
sys.path[0] = MANTIDIMAGING_PATH

import mantidimaging
print("Package name ->", mantidimaging.__package__)

Home Dir -> /home/dtasev 
Dataset Path -> /home/dtasev/win_img/chadwick_insert 
Package Path -> /home/dtasev/mantidimaging
Package name -> mantidimaging


In [6]:
import matplotlib

# Setup matplotlib to use the notebook backend, this will allow us to show images/histograms in here
matplotlib.use('nbagg')

from matplotlib import pyplot

<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>
# Loading data

In [7]:
# Load some data in, format for indices is [start, end, step]
indices = [0, 1570, 100]
images = mantidimaging.io.loader.load(CHADWICK_SAMPLE_PATH, CHADWICK_FLAT_PATH, CHADWICK_DARK_PATH, indices=indices)

In [8]:
# The file returned is of a type Images, it allows us to transfer metadata with the images, 
# such as the filenames. The filenames are used in the GUI to show the current slice's original file name
assert isinstance(images, mantidimaging.io.loader.Images)

# print the public methods
list(filter(lambda x: "__" not in x, dir(isis_imaging.io.loader.Images)))

NameError: name 'isis_imaging' is not defined

In [ ]:
# Use matplotlib to build a small plotting window that visualises the image and a histogram
def show(image):
    fig, img_axes = pyplot.subplots(nrows=1,ncols=1)
    
    image_obj = img_axes.imshow(image.get_sample()[0], cmap='Greys_r')
    
    hist_axes = fig.add_axes([0.25, 0.925, 0.5, 0.08])
    hist_axes.hist(image.get_sample()[0].flatten(), bins=2048)

    pyplot.show()
    
# Due to this being added at a later time, show was already used a few times and I didn't want to break them
# This uses the image argument directly instead of accessing the .get_sample() method
def show2(image):
    fig, img_axes = pyplot.subplots(nrows=1,ncols=1)
    
    image_obj = img_axes.imshow(image, cmap='Greys_r')
    
    hist_axes = fig.add_axes([0.25, 0.925, 0.5, 0.08])
    hist_axes.hist(image.flatten(), bins=2048)

    pyplot.show()

In [ ]:
show(images)

# Flat and Dark images are usually between 30-60, 
# but we load all of them in and then take the average of all the images

# Flat are also called Open Beam and are images that are 
# run after the instrument/experiment is set up, but without the actual sample
show2(images.get_flat())

# Dark images are gathered from running the camera with a 
# closed shutter, showing any bad pixels from the camera itself
show2(images.get_dark())

<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>
# Background Correction

In [ ]:
from mantidimaging.core.algorithms import value_scaling
from mantidimaging.core.filters import background_correction

# Calculate scale factors, so that we can scale up the values 
# after ROI normalisation (more details in next step)
scale_factors = value_scaling.create_factors(images.get_sample())

# Background Correction
# We need sample, flat (open beam without an object) and dark (closed shutter) images
background_correction.execute(images.get_sample(), images.get_flat(), images.get_dark())

# show result
show(images)

<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>
# Some images might have slightly lower/higher values, due to changes in the flux

In [ ]:
from mantidimaging.gui.stack_visualiser import sv_histogram

# Show difference in histograms, but we don't want to look at two different plots
sv_histogram.show_transparent(images.get_sample()[0].flatten(), "Normal Contrast", "")
sv_histogram.show_transparent(images.get_sample()[5].flatten(), "Low Contrast", "")

# But we don't really want to look at 2 different histograms

In [ ]:
# import package from isis_imaging's GUI part
from mantidimaging.gui.stack_visualiser import sv_histogram

# python abuse to plot the histograms on the same plot, it works by default in the package

def plot_histograms_magically():
    # save the original function reference
    temp_func_storage = sv_histogram._show
    def _temporary_empty_function(): pass
    
    # set to a function that does nothing
    sv_histogram._show = _temporary_empty_function

    sv_histogram.show_transparent(images.get_sample()[0].flatten(), "Normal Contrast", "")

    # restore so that the plot is shown next time
    sv_histogram._show = temp_func_storage

    sv_histogram.show_transparent(images.get_sample()[5].flatten(), "Low Contrast", "Magic")

In [ ]:
plot_histograms_magically()

<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>
# Normalisation by Region of Interest

In [ ]:
AIR_REGION = (1348, 732, 1436, 846)
print("Air Region", *AIR_REGION)



# Normalisation by Air Region (Contrast Normalisation)

from isis_imaging.core.filters import roi_normalisation

# Normalise by region of interest
roi_normalisation.execute(images.get_sample(), AIR_REGION)

# and scale up the values to uint16 range (0-65535)
value_scaling.apply_factor(images.get_sample(), scale_factors)

In [ ]:
show(images)

In [ ]:
plot_histograms_magically()

<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>
# Region of Interest crop

In [ ]:
ROI = (422, 6, 1492, 1970)
print("ROI", *ROI)

from mantidimaging.core.filters import crop_coords

print("The Region of Interest that we are cropping", ROI)
images.sample, images.flat, images.dark = crop_coords.execute(images.get_sample(), ROI)

In [ ]:
show(images)

In [ ]:
plot_histograms_magically()

## We have not removed _any_ of the data!
## But the noise counts are now a lot less
### Before: 120,000
### Now: 42,000

<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>
# Reducing the noise further

### Values of noise and data are very distinct

In [ ]:
show(images)
plot_histograms_magically()

In [ ]:
CLIP_MIN = 0.01
CLIP_MAX = 6200

from mantidimaging.core.filters import clip_values

print("Remove minimum values", CLIP_MIN)
print("Remove maximum values", CLIP_MAX)
clip_values.execute(images.get_sample(), CLIP_MIN, CLIP_MAX)

In [ ]:
show(images)

In [ ]:
plot_histograms_magically()

In [ ]:
from mantidimaging.core.filters import minus_log

# Invert the histogram and reduce the range
images.sample = minus_log.execute(images.get_sample())

In [ ]:
show(images)

In [ ]:
plot_histograms_magically()